In [1]:
# @Author: Apeksha
# Approach: Using sklearn MultiOutputClassifier generating multiple label output.
# Step: 1. Read the processed label and features.
#       2. Apply PCA, REG, Normalization, Scaling --- TODO
#       3. Prepare data for Cross Validation 
#       4. Do Classification
#       5. Evaluation



In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [3]:
#Method to read dataset
def read_dataset(path):
    return pd.read_csv(path,header="infer",sep=",",low_memory=False)

In [4]:
#set path to read the processed features files

train_path=r"B:\MS_SCU\Winter20\DataMiningAndPatterRecognition\Project\train1.csv"
train_label_path=r"B:\MS_SCU\Winter20\DataMiningAndPatterRecognition\Project\train_labels.csv"
train_df=read_dataset(train_path)
train_label_df=read_dataset(train_label_path)

test_path=r"B:\MS_SCU\Winter20\DataMiningAndPatterRecognition\Project\test.csv"
test_label_path=r"B:\MS_SCU\Winter20\DataMiningAndPatterRecognition\Project\test_label.csv"
test_df=read_dataset(test_path)
test_label_df=read_dataset(test_label_path)

In [5]:
# with open("output_train_columns.txt", 'w') as filehandle:
#     filehandle.writelines("%s\n" % col for col in train_df.columns)
    
# with open("output_test_columns.txt", 'w') as filehandle:
#     filehandle.writelines("%s\n" % col for col in test_df.columns)

In [6]:
print('Train df shape: ',train_df.shape)
print('Train label df shape: ', train_label_df.shape)
print('test df shape: ',test_df.shape)
print('test label df shape: ',test_label_df.shape)

Train df shape:  (82717, 105)
Train label df shape:  (82717, 27)
test df shape:  (18278, 105)
test label df shape:  (18278, 27)


In [7]:
# Method to train the model
# Params- Estimator: Classifier being used for classification
#         X_train: input features
#         Y_train: input labels
#Return- Trained model of type Estimator 
def fit_model(estimator,X_train, y_train):
    print('Inside fit_model', X_train.shape,' labels: ',y_train.shape)
    return MultiOutputClassifier(estimator).fit(X_train, y_train)

# Method to make the predictions
# Params- Model: Trained model
#         X_test: Test data
#Return-  A list of labels per record. [[],[],[],[]] of shape 82117 * 27
def predict_poistion(model,X_test):
    print('Inside prediction: ',X_test.shape)
    return model.predict(X_test)

# Method to get model evaluation paramaters
def print_classification_report(predictions,actual_labels):
    print("actual_labels: ",actual_labels.shape)
    print(classification_report(actual_labels, predictions))
    
# Method to write predictions to a text file.
def generate_output_file(predictions):
    output_path="output.txt"
    with open(output_path, 'w') as filehandle:
        filehandle.writelines("%s\n" % label for label in predictions)
        
def apply_postprocessing(model_name,pred_df):
    #Following line of code based off of Apeksha's Code From Feature Extraction
    labels=pred_df[pred_df.apply(lambda x: x == pred_df.max(axis = 1))] 
    #print(labels)
    classes=[]

    #Write all labels to this file
    outputFile = open(model_name+'.txt','w')

    for i in range(labels.shape[0]):
        #Following line of code based off of Apeksha's Code From Feature Extraction
        l=list(labels.columns[np.isfinite(labels.iloc[i])])
        #print(l)
        formatted_l=map(lambda x:x+' ', l)
        #Eventually add in the players's name too
        outputFile.writelines("ID: " + str(i) + ", Total Classes: " + str(len(l)) + " Classes: ")
        outputFile.writelines(formatted_l)
        outputFile.writelines('\n')
        #for i in l:
        #    outputFile.write(i)
        #    outputFile.write('\n')
        classes.append(l)
        #Save the classes to a file
    #print(len(classes))
    #print(classes)
    outputFile.close()

In [8]:
#A model definition for cross validation.
all_models=[]

In [9]:
#Logistic regression Estimator
lr_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,penalty='l2', random_state=None, solver='liblinear', tol=0.0001,verbose=0, warm_start=False)
lr_model=fit_model(lr_estimator,train_df, train_label_df)
all_models.append(("LR",lr_model))

lr_predictions=predict_poistion(lr_model,test_df)
#print_classification_report(lr_predictions,test_label_df)

lr_pred_df = pd.DataFrame(lr_predictions, columns = ['gk', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'])
apply_postprocessing("lr",lr_pred_df)

Inside fit_model (82717, 105)  labels:  (82717, 27)
Inside prediction:  (18278, 105)


In [10]:
from sklearn.metrics import f1_score
lr_f1_score=f1_score(test_label_df,lr_predictions,average='weighted')
lr_f1_score

0.7268794064981532

In [11]:
#SVM Estimator
# sv_estimator=SVC()

# sv_model=fit_model(sv_estimator,train_df, train_label_df)
# all_models.append(("SVM",sv_model))

# sv_predictions=predict_poistion(sv_model,test_df)
# print_classification_report(sv_predictions,test_label_df)

# sv_pred_df = pd.DataFrame(sv_predictions, columns = ['gk', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'])
# apply_postprocessing("SVM",sv_pred_df)
 

In [12]:
# Random Forest Estimator
rf_estimator=RandomForestClassifier(n_estimators=100,bootstrap = True,max_features = 'sqrt')

rf_model=fit_model(rf_estimator,train_df, train_label_df)
all_models.append(("RF",rf_model))
rf_predictions=predict_poistion(rf_model,test_df)

#print_classification_report(rf_predictions,test_label_df)
rf_pred_df = pd.DataFrame(rf_predictions, columns = ['gk', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'])
apply_postprocessing("RF",rf_pred_df)

Inside fit_model (82717, 105)  labels:  (82717, 27)
Inside prediction:  (18278, 105)


In [13]:
from sklearn.metrics import f1_score
rf_f1_score=f1_score(test_label_df,rf_predictions,average='weighted')
rf_f1_score

0.8262074153177804

In [14]:
#KNN Estimator
knn_estimator=KNeighborsClassifier(n_neighbors=3)

knn_model=fit_model(knn_estimator,train_df, train_label_df)
all_models.append(("KNN",knn_model))
knn_predictions=predict_poistion(knn_model,test_df)

#print_classification_report(knn_predictions,test_label_df)
knn_pred_df = pd.DataFrame(knn_predictions, columns = ['gk', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'])
apply_postprocessing("knn",knn_pred_df)


Inside fit_model (82717, 105)  labels:  (82717, 27)
Inside prediction:  (18278, 105)


In [15]:
from sklearn.metrics import f1_score
knn_f1_score=f1_score(test_label_df,knn_predictions,average='weighted')
knn_f1_score

0.7201385690253673

In [20]:
#https://stackoverflow.com/questions/28200786/how-to-plot-scikit-learn-classification-report
all_models=[("LR",lr_model),("RF",rf_model),("KNN",knn_model)]
all_models

[('LR',
  MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                     dual=False,
                                                     fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None, max_iter=100,
                                                     multi_class='ovr', n_jobs=1,
                                                     penalty='l2',
                                                     random_state=None,
                                                     solver='liblinear',
                                                     tol=0.0001, verbose=0,
                                                     warm_start=False),
                        n_jobs=None)),
 ('RF', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                         cl

In [ ]:
# @Author: Apeksha
# Method to use for Cross Validation of all the models. 
# This method is used for 10 fold cross validation of entire train set.
# @Params: 
#         all_model: A list of tuple of the form (<'model name'>,<'model object'>)
#
from sklearn import model_selection
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name,model in all_models:
    print("rounds: :::::::")
    kfold = model_selection.KFold(n_splits=10, random_state=0.11)
    cv_results = model_selection.cross_val_score(model, train_df, train_label_df, cv=kfold, scoring=scoring)
    print(cv_results)
    results.append(np.average(cv_results))
    names.append(name)    
compare_f1_scores(names,results)

rounds: :::::::
[0.54545455 0.55560928 0.60287718 0.62632979 0.61496615 0.62814313
 0.61762573 0.6261637  0.62422923 0.62628461]
rounds: :::::::


In [ ]:
#print(results[3])
bpr=[lr_f1_score,rf_f1_score,knn_f1_score]

In [ ]:
ls=[0.54545455, 0.55560928, 0.60287718, 0.62632979, 0.61496615, 0.62814313, 0.61762573, 0.6261637,  0.62422923, 0.62628461]
print(np.average(ls))

In [ ]:
%matplotlib inline
# boxplot algorithm comparison
import matplotlib.pyplot as plt
model_names=["LR","RF","KNN"]
f1_scores=[lr_f1_score,rf_f1_score,knn_f1_score]
def compare_f1_scores(model_names,f1_scores):
    y_pos = np.arange(len(model_names))
    plt.bar(y_pos, f1_scores, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('F1 Scores')
    plt.title('Classifiers')
    plt.show()

In [ ]:
compare_f1_scores(model_names,f1_scores)

In [ ]:
knn_predictions

In [ ]:
score_df=pd.DataFrame(data=f1_scores,columns=model_names)
score_df=score_df.mul([1, 100], axis=0)
score_df=score_df.round(2)